**The Leading Edge - Geophysical Tutorial (TLE October 2017)** 

# Coloured Inversion Workflow

Martin Blouin$^1$ and Erwan Gloaguen$^2$

Whether it is deterministic, band-limited or stochastic, seismic inversion can bear many names depending on the process used to achieve it and on the purpose for which it is designed.  Basically, a seismic inversion is the process where reflectivity data is converted to ground mechanical properties, such as seismic impedance that is function of seismic velocity and density (see for example [Francis (2014)](#References) for a comprehensive and quick overview of seismic inversion options). Reflectivity informs on boundaries but impedance data are commonly used as they can be converted to informative ground property such as porosity and fluid content through known petrophysical relations.

Following the work of [Lancaster and Whitcombe (2000)](#References), a fast method for providing band limited inversion of seismic data known as Coloured Inversion (CI) generated wide spread interest amongst interpreters in the Oil & Gas industry. The idea was born from the acknowledgement that sparse spike inversion process could be approximated by a single operator. The authors showed that this operator can be derived from well logs and yield relative impedance only by a simple convolution with the reflectivity data. Its popularity can be attributed to many factors: the simple workflow, the fast computation and the calculation of an operator instead of an explicit wavelet estimation. This makes the interpreter rely less on a specialist at an early stage (before more complex inversion are run on the data) and base is analysis on tangible and measurable well log information. With comparable result to sparse spiked inversion, CI can enhance features such as bed resolution and discontinuities. Since CI is directly linked to seismic data, the relative AI produced from it can be used as a base for comparison with other inversion to see what kind of information is introduced by constraints and *a priori* model.

In this tutorial, we will follow the steps presented by Lancaster and Whitcombe in their 2000 Expanded abstract to achieve the so called « fast-track coloured inversion » and describe all the steps with open-source algorithms to go from reflectivity data to inversed cubes.

1.	Fit a function to the log spectra
2.	Get difference spectra by substracting the seismic spectra from it
3.	Convert the difference spectra in an operator
4.	Convolute the stacked seismic with the operator
5.	QC – check the residuals by comparing log and AI section spectra

The idea of this tutorial is to achieve Coloured Inversion without any external program, just python code with `numpy` and `scipy` library. The tutorial focus on presenting the whole workflow in the simplest fashion rather than trying to optimize parameters to recover interpretable features. Prior to running the workflow on your data, noise attenuation should be performed to ensure that the inversion process recovers frequencies associated only with the Earth signal.

## The dataset

For this example, we use the dGB Earth Sciences, OpendTect F3 Dataset from which we recovered one well AI log and a vertical seismic 2D panel collocated with the F02-1 well log. F3 is a block in the Dutch sector of the North Sea. The block is covered by 3D seismic that was acquired to explore for Oil & Gas which are found below the interval selected for this demo set. The upper part ($<$1200ms) of this demo set consists of reflectors from the Miocene, Pliocene, and Pleistocene. Good descriptions of the geological settings for this dataset can be found in [Sorensen et al, 1997](#References) and [Overeem et al, 2001](#References)). We use the dip-steered median filter stacked dataset to get reduced noise on our input. A 2D panel of the survey at Inline=362 is presented in Figure 1.

![Figure 1](Figures/seismic_inline362.png "Figure 1")
<center>*Figure 1. F3 dip-steered median filtered stacked seismic data at Inline=362.*</center>

## Approximating the well spectra

One of the key point in the development of this approach, relies in the fact that it is computationally inexpensive and easy to understand originated from the observations that the relfectivity sequence in sedimentary environments follows a logarithmic decay in amplitude as frequency increases. For the first step of our inversion, we need to look at the spectra and make sure it behaves as predicted by [Walden and Hosken (1984)](#References). We load the F02-1 well and calculate the spectra after converting it to time domain.
```python
n_log = AI_f021.shape[0]
k_log=np.arange(n_log)
T_log = 1000/depth_f021 # converting from ms to s
freq_log = k_log/T_log
freq_log = freq_log[range(n_log//2)] # one side frequency range
spec_log = np.fft.fft(AI_f021)/n_log # fft computing and normalization
spec_log = spec_log[range(n_log//2)]
```
Figure 2 shows the F02-1 AI log and the associated calculated spectra.

![Figure 2](Figures/f021_log_spectra.png "Figure 2")
<center>*Figure 2. F02-1 well data. a) AI Log, b) Calculated frequency power spectra.*</center>

To make the process more robust, CI packages offers you the option of averaging this spectrum over all the available wells before proceeding to any regression. In this example, as a demonstration, we are working only with a 2D section so we proceed only with the F02-1 which is collocated with this section. In order to proceed to a linear regression, we define two simple functions, one that serves as the linearization of the problem and one for the error function. I then use `scipy.optimize` to find the best fit.
```python
def my_fun(p,x):
    return p[0]*x**p[1]
    
def err_fun(p,x,y):    
    return (np.log10(y) - np.log10(my_fun(p,x)))

qout,success = optimize.leastsq(err_fun, [1e5,-0.8],args=(freq_log[1:],np.abs(spec_log[1:])),maxfev=3000)
```
Figure 3 presents the spectra in log space with the linear function to approximate it. 

![Figure 3](Figures/Well_spectra_approx.png "Figure 3")
<center>*Figure 3. F02-1 well spectra approximated by a linear function in log space*</center>

## From difference spectra to the operator

Now we have a continuous function that is now mathematically approximating the well log spectra permitting the seismic data to have the same frequency content. The next major step of the workflow is to get an operator that is representative of the difference between the two spectras. We first define some boundaries to our modelled spectra in the frequency domain. This is a critical part that will have great influence on the end result. We define a simple function to generate a 50 points Hanning shaped taper at both end of the spectra.
```python
x_tape_in = np.linspace(0,5,50)
x_tape_out = np.linspace(100,120,50)
id_seis = (freq_seis > 5) * (freq_seis < 100)
y_tape_in = np.hanning(100)[:50]*my_fun(qout,x_tape_in[-1])
y_tape_out = np.hanning(100)[50:]*my_fun(qout,x_tape_out[0])
new_freq_log = np.hstack([x_tape_in,freq_seis[id_seis],x_tape_out])
new_spec_log = np.hstack([y_tape_in,my_fun(qout,freq_seis[id_seis]),y_tape_out])
```
Now we take seismic traces next to the well location, averaged them and repeat the procedure we did for the well to get the spectra. Figure 4 presents the resulting modelled spectra with the underlying well log and seismic spectra. In this example, taper windows of 0-5 Hz and 100-120 Hz have been defined. It is important to note that our end result will be greatly influenced by this choice and care should be applied at this stage.

![Figure 4](Figures/All_spectra.png "Figure 4")
<center>*Figure 4. Comparison of well log, seismic and modelled frequency power spectra*</center>

After this step is completed, we simply substract the seismic spectra from our modelled spectra, take the resulting spectra, do a Fourier backtransform and do a phase rotation. 
```python
operator = np.fft.ifft(gap)
operator = np.fft.fftshift(operator)
operator = operator.imag
```
Figure 5a presents the difference spectra and Figure 5b shows the resulting operator.

![Figure 5](Figures/diff_spec_and_operator.png "Figure 5")
<center>*Figure 5. a) Resulting difference spectra b) Associated Operator*</center>

## Impedance result

Once the operator is calculated a simple trace by trace convolution with the reflectivity data is needed to perform coloured inversion.
```python
convo= np.array([np.convolve(panel_seis[:,col],operator,mode='same') for col in range(len(panel_seis[0,:]))])
```
Since the outputted relative impedance does not inform us about trends, it makes more sense to look at the result in a single formation. Indeed, both vertical and lateral trends are defined by a low frequency model and not directly by the seismic. It would be expected, for example, for impedance to increase as we go deeper and rock formations become stiffer. Here, it wouldn’t be the case if we look at the full vertical AI section. So instead, we will focus on an area of interest of about 300 ms in thickness between two horizons. Figure 6 presents the resulting relative impedance section scaled (0-1) in this area.

![Figure 6](Figures/Relative_impedance_result.png "Figure 6")
<center>*Figure 6. Relative impedance result of the CI workflow*</center>

## A simple QC procedure

To make sure that we did a good job defining our operator, we now look at the resulting spectra from the relative impedance collocated with the well log and see if we achieved a good fit with the well log spectra (Figure 7). It is to be noted that this fit was achieved by a manual fit and the workflow could be integrated in an iterative process to minimize the difference between the two spectra.

![Figure 7](Figures/QC.png "Figure 7")
<center>*Figure 7. QC of th CI workflow with comparison of input and output spectra*</center>

## Conclusion

In this tutorial, we presented a straight forward application of coloured inversion using python with only `numpy` and `scipy` librairies. We demonstrate that this open-source workflow is simple and can yield informative images to interpreters really fast. It is called a robust process in the literature althought we saw it is somewhat sensitive to the chosen frequency range. There are also choices to be made about the window of application and the number of traces for spectra calculation. An important thing to understand about inversion such as CI is that it cannot be informative of trends.

Corresponding author: martin.blouin@geolearn.ca

$^1$ GeoLEARN Solutions, Quebec City
$^2$ INRS-ETE, Quebec City

## References

**Francis, A., 2014**, A simple guide to seismic inversion: GEOExPro, 10.2, 46-50.[Link to article](http://www.geoexpro.com/articles/2014/06/a-simple-guide-to-seismic-inversion)  

**Overeem, I, G. J. Weltje, C. Bishop-Kay, and S. B. Kroonenberg, 2001**, The Late Cenozoic Eridanos delta system in the Southern North Sea Basin: a climate signal in sediment supply? Basin Research, 13, 293–312.
DOI: [10.1046/j.1365-2117.2001.00151.x](http://onlinelibrary.wiley.com/doi/10.1046/j.1365-2117.2001.00151.x)

**Sørensen, J.C., Gregersen, U, Breiner, M and Michelsen, O., 1997**, High frequency sequence stratigraphy of upper Cenozoic deposits: Mar. Petrol. Geol., 14, 99–123.  
DOI: [10.1016/S0264-8172(96)00052-9](https://doi.org/10.1016/S0264-8172)  
   
**Walden, A.T., and Hosken, J.W.J., 1985**, An investigation of the spectral properties of primary reflection coefficients: Geophysical Prospecting , 33, 400-435. 
DOI: [10.1111/j.1365-2478.1985.tb00443.x](http://onlinelibrary.wiley.com/doi/10.1111/j.1365-2478.1985.tb00443.x/)  

&copy; Martin Blouin. Licensed CC-BY-SA. Final article published by the Society of Exploration Geophysicists in **The Leading Edge**, October 2017.

Citation:  Blouin, M and Gloaguen, E., 2017, Coloured Inversion Workflow: The Leading Edge,XX, XXX-XXX;
doi: [XXX](http://dx.doi.org/xxx).